## 1.安裝套件

In [3]:
!pip install finlab > log.txt

## 2.本益成長比策略

[PEG策略教學](https://www.finlab.tw/finlab-tw-stock-peg-strategy/)

In [4]:
from finlab import data
from finlab.backtest import sim

rev = data.get('monthly_revenue:當月營收')
rev_ma3 = rev.average(3)
rev_ma12 = rev.average(12)

# 營收趨勢多頭策略
cond1 = rev_ma3/rev_ma12 > 1.1
cond2 = rev/rev.shift(1) > 0.9
cond_all = cond1 & cond2

pe = data.get('price_earning_ratio:本益比')
營業利益成長率 = data.get('fundamental_features:營業利益成長率')

# 本益成長比
peg = (pe/營業利益成長率)

# 本益成長比和原訊號相乘，若不持有則相乘後等於0
position = peg*(cond_all)

# 原訊號為0的不要選，若沒加這行且策略只選到7檔，之後還是會選3檔訊號為0(不持有)的補足10檔，執行這行就會排除訊號為0。
position = position[position>0]

# 選股挑本益成長比前10小的
position = position.is_smallest(10)

## 3.月營收公告截止日換股

position的dataframe用營收的index，不要用收盤價的index。

## note:

position加上[reindex](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reindex.html)月營收的日期索引，後會讓position日期變成每月營收截止日。

resample不須另外設定，使用預設"None"，不執行再平衡。position想成是每列而不是每日，因為使用reindex讓每列間隔一個月，所以執行上是隔一個月產生換股訊號。

In [5]:
# 月營收截止日換股
position = position.reindex(rev.index_str_to_date().index, method='ffill')

report = sim(position=position, name="策略教學範例:peg_rev", stop_loss=0.1, upload=False)
report.display()

1451    0.1
1475    0.1
1584    0.1
2612    0.1
2641    0.1
2702    0.1
3706    0.1
5432    0.1
6199    0.1
9906    0.1
Name: 2022-08-10 00:00:00, dtype: float64

Timestamp('2022-08-10 00:00:00')